# Testing Each Data Stream

<br>

<b>Data Streams:</b>

* Generation by fuel type 
* Forecast demand (*Need to add)
* Forecast wind and solar
* Realised wind and solar 
* Imbalance price and volume
* Predicted demand and imbalance volumes
* Imbalance market bids and offers 
* Accepted bids and offers 


<br>

### Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, date

import seaborn as sns
import matplotlib.pyplot as plt
    
import BMRS_caller as BMRS
import warnings

<br>

### Report Grabber Orchestrator

In [2]:
## Could also add in default kwargs
## Could combine with stream_url_dict


reports_2_grab = {
    'FUELHH' : {
        'name' : 'Fuel Half-Hour',
        'final_cols' : ['ccgt', 'oil', 'coal', 'nuclear', 'wind', 'ps', 'npshyd', 'ocgt', 'other', 'intfr', 'intirl', 'intned', 'intew', 'biomass', 'intnem'],
    },
      
    'B1440' : {
        'name' : 'Day-Ahead Generation forecasts for Wind and Solar',
        'long_2_wide' : {
            'index' : 'local_datetime',
            'column' : 'powerSystemResourceType',
            'values' : 'quantity'
        },
    },
    
            
    'B1630' : {
        'name' : 'Actual Or Estimated Wind and Solar Power Generation',
        'long_2_wide' : {
            'index' : 'local_datetime',
            'column' : 'powerSystemResourceType',
            'values' : 'quantity'
        },
    },
    
    'B1770' : {
        'name' : 'Imbalance Prices',
        'long_2_wide' : {
            'index' : 'local_datetime',
            'column' : 'priceCategory',
            'values' : 'imbalancePriceAmountGBP'
        },
    },
    
    'B1780' : {
        'name' : 'Aggregated Imbalance Volumes',
        
    },
    
    'MELIMBALNGC' : {
        'name' : 'Forecast Day and Day Ahead Margin and Imbalance',
        'final_cols' : ['nationalBoundaryIdentifier', 'margin', 'imbalanceValue'],
    },
    
    
}

In [3]:
def clean_df(df, report_dict):
    ## Reformatting Dataframe
    if 'long_2_wide' in report_dict.keys():
        long_2_wide_dict = report_dict['long_2_wide']
        df = df.pivot(long_2_wide_dict['index'], long_2_wide_dict['column'], long_2_wide_dict['values'])
    else:
        df = df.set_index('local_datetime')

    ## Filtering columns
    if 'final_cols' in report_dict.keys(): 
        df = df[report_dict['final_cols']]

    return df

<br>

### Testing Data Streams

<b>User Inputs</b>

In [4]:
API_key = 'your_API_key'

<br>

<b>Half-hourly generation by fuel type</b>

In [5]:
report = 'FUELHH'
start_date = date(2018, 1, 1)
end_date = datetime(2018, 1, 2, 23, 30)

Caller = BMRS.Caller(API_key, report)
df_raw = Caller.call(start_date=start_date, end_date=end_date)
df = clean_df(df_raw, reports_2_grab[report])

df.head(2)

,ccgt,oil,coal,nuclear,wind,ps,npshyd,ocgt,other,intfr,intirl,intned,intew,biomass,intnem
local_datetime,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,5361,0,1237,7267,8596,0,353,0,89,1408,206,710,432,749,0
2018-01-01 00:30:00+00:00,5739,0,1404,7265,8750,0,353,0,89,1404,200,710,432,756,0


In [6]:
df.tail(2)

,ccgt,oil,coal,nuclear,wind,ps,npshyd,ocgt,other,intfr,intirl,intned,intew,biomass,intnem
local_datetime,,,,,,,,,,,,,,,
2018-01-02 23:00:00+00:00,8352,0,1133,7263,7102,0,467,0,91,1606,80,746,382,681,0
2018-01-02 23:30:00+00:00,7561,0,1149,7263,7189,0,444,0,91,1600,82,700,432,681,0


<br>

<b>Forecasted half-hourly generation for renewable sources type</b>

In [7]:
report = 'B1440'
start_date = date(2018, 1, 1)
end_date = datetime(2018, 1, 2, 23, 30)

Caller = BMRS.Caller(API_key, report)
df_raw = Caller.call(start_date=start_date, end_date=end_date)
df = clean_df(df_raw, reports_2_grab[report])

df.head(2)

powerSystemResourceType,"""Solar""","""Wind Offshore""","""Wind Onshore"""
local_datetime,,,
2018-01-01 00:00:00+00:00,0,4835.335,5870.158
2018-01-01 00:30:00+00:00,0,4835.335,5870.158


In [8]:
df.tail(2)

powerSystemResourceType,"""Solar""","""Wind Offshore""","""Wind Onshore"""
local_datetime,,,
2018-01-01 23:30:00+00:00,0,4878.575,5259.375
2018-01-02 00:00:00+00:00,0,4672.458,5207.216


<br>

<b>Realised half-hourly generation for renewable sources type</b>

In [9]:
report = 'B1630'
start_date = date(2018, 1, 1)
end_date = datetime(2018, 1, 2, 23, 30)

Caller = BMRS.Caller(API_key, report)
df_raw = Caller.call(start_date=start_date, end_date=end_date)
df = clean_df(df_raw, reports_2_grab[report])

df.head(2)

powerSystemResourceType,"""Solar""","""Wind Offshore""","""Wind Onshore"""
local_datetime,,,
2018-01-01 00:00:00+00:00,0,3777.006,4950.296
2018-01-01 00:30:00+00:00,0,3824.889,5004.552


In [10]:
df.tail(2)

powerSystemResourceType,"""Solar""","""Wind Offshore""","""Wind Onshore"""
local_datetime,,,
2018-01-01 23:30:00+00:00,0,3495.327,4137.181
2018-01-02 00:00:00+00:00,0,3335.834,4160.384


<br>

<b>Imabalance Market Prices</b>

In [11]:
report = 'B1770'
start_date = date(2018, 1, 1)
end_date = date(2018, 1, 2)

Caller = BMRS.Caller(API_key, report)
df_raw = Caller.call(start_date=start_date, end_date=end_date)
df = clean_df(df_raw, reports_2_grab[report])

df.head(2)

priceCategory,Excess balance,Insufficient balance
local_datetime,,
2018-01-01 00:00:00+00:00,55.94,55.94
2018-01-01 00:30:00+00:00,55.94,55.94


In [12]:
df.tail(2)

priceCategory,Excess balance,Insufficient balance
local_datetime,,
2018-01-01 23:30:00+00:00,24.67,24.67
2018-01-02 00:00:00+00:00,23.06323,23.06323


<br>

<b>Imabalance Market Volumes</b>

In [13]:
report = 'B1770'
start_date = date(2018, 1, 1)
end_date = date(2018, 1, 2)

Caller = BMRS.Caller(API_key, report)
df_raw = Caller.call(start_date=start_date, end_date=end_date)
df = clean_df(df_raw, reports_2_grab[report])

df.head(2)

priceCategory,Excess balance,Insufficient balance
local_datetime,,
2018-01-01 00:00:00+00:00,55.94,55.94
2018-01-01 00:30:00+00:00,55.94,55.94


In [14]:
df.tail(2)

priceCategory,Excess balance,Insufficient balance
local_datetime,,
2018-01-01 23:30:00+00:00,24.67,24.67
2018-01-02 00:00:00+00:00,23.06323,23.06323


<br>

<b>Forecast Day and Day Ahead Margin and Imbalance</b>

In [15]:
## Should look into adding a way to pivot on the NBI and split margin and imbalance value

report = 'MELIMBALNGC'
start_date = date(2018, 1, 1)
end_date = datetime(2018, 1, 2, 23, 30)
zone = '*'

Caller = BMRS.Caller(API_key, report)
df_raw = Caller.call(start_date=start_date, end_date=end_date, zone=zone)
df = clean_df(df_raw, reports_2_grab[report])

df.head(2)

,nationalBoundaryIdentifier,margin,imbalanceValue
local_datetime,,,
2018-01-01 00:00:00+00:00,B1,-740,NaN
2018-01-01 00:00:00+00:00,B2,-494,NaN


In [16]:
df.tail(2)

,nationalBoundaryIdentifier,margin,imbalanceValue
local_datetime,,,
2018-01-02 23:30:00+00:00,B16,NaN,3869
2018-01-02 23:30:00+00:00,B17,NaN,-3452
